In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('myApp').getOrCreate()

In [3]:
from pyspark.sql.types import (IntegerType,StringType,StructField,StructType)

In [4]:
mylist = [[1, 2, 3, 4,5],[4,5,6,8,9],[7,8,5,4,4],[3,6,4,8,6],[5,7,6,2,1]]
#mylist = [5,7,6,2,1]

In [6]:
#spark.createDataFrame(mylist,IntegerType()).show()

In [7]:
cSchema = StructType([StructField("a", IntegerType()),StructField("b", IntegerType()),StructField("c", IntegerType()),StructField("d", IntegerType()),StructField("e", IntegerType())])

In [8]:
#spark.createDataFrame(mylist,IntegerType()).show()
df = spark.createDataFrame(mylist,schema=cSchema) 

In [9]:
df.show()

+---+---+---+---+---+
|  a|  b|  c|  d|  e|
+---+---+---+---+---+
|  1|  2|  3|  4|  5|
|  4|  5|  6|  8|  9|
|  7|  8|  5|  4|  4|
|  3|  6|  4|  8|  6|
|  5|  7|  6|  2|  1|
+---+---+---+---+---+



In [10]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window as W

In [11]:
df.withColumn('TOTAL', df.a + df.b + df.c + df.d+ df.e).show()

+---+---+---+---+---+-----+
|  a|  b|  c|  d|  e|TOTAL|
+---+---+---+---+---+-----+
|  1|  2|  3|  4|  5|   15|
|  4|  5|  6|  8|  9|   32|
|  7|  8|  5|  4|  4|   28|
|  3|  6|  4|  8|  6|   27|
|  5|  7|  6|  2|  1|   21|
+---+---+---+---+---+-----+



In [12]:
newDf =df.withColumn('TOTAL', df.a + df.b + df.c + df.d+ df.e)

In [13]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [14]:
my_window = Window.partitionBy().orderBy("a")

df1 = newDf.withColumn("prev_value", F.lag(newDf.TOTAL).over(my_window))

df1.show()

+---+---+---+---+---+-----+----------+
|  a|  b|  c|  d|  e|TOTAL|prev_value|
+---+---+---+---+---+-----+----------+
|  1|  2|  3|  4|  5|   15|      null|
|  3|  6|  4|  8|  6|   27|        15|
|  4|  5|  6|  8|  9|   32|        27|
|  5|  7|  6|  2|  1|   21|        32|
|  7|  8|  5|  4|  4|   28|        21|
+---+---+---+---+---+-----+----------+



In [20]:
df1 = df1.withColumn("diff", F.when(F.isnull(df1.TOTAL - df1.prev_value), 0)
                             .when((df1.TOTAL==32), 0)
                              .otherwise(df1.TOTAL + df1.prev_value))


In [21]:
df1.show()

+---+---+---+---+---+-----+----------+----+
|  a|  b|  c|  d|  e|TOTAL|prev_value|diff|
+---+---+---+---+---+-----+----------+----+
|  1|  2|  3|  4|  5|   15|      null|   0|
|  3|  6|  4|  8|  6|   27|        15|  42|
|  4|  5|  6|  8|  9|   32|        27|   0|
|  5|  7|  6|  2|  1|   21|        32|  53|
|  7|  8|  5|  4|  4|   28|        21|  49|
+---+---+---+---+---+-----+----------+----+

